In [2]:
import shutil
import os

# Directory to delete
EXTRACT_DIR = 'test'

# Check if the directory exists and delete it
if os.path.isdir(EXTRACT_DIR):
    shutil.rmtree(EXTRACT_DIR)
    print(f"Directory '{EXTRACT_DIR}' has been deleted.")
else:
    print(f"Directory '{EXTRACT_DIR}' does not exist.")

Directory 'test' has been deleted.


In [3]:
import zipfile
import shutil
import os
import pandas as pd
print("CPU cores: ", os.cpu_count())

CPU cores:  144


In [4]:
def extract_zip(zip_path):

    if not zipfile.is_zipfile(zip_path):
        raise ValueError(f"The file at {zip_path} is not a valid ZIP archive.")
    
    # Determine the output directory name from the zip file name
    base_dir = os.path.dirname(zip_path)
    zip_filename = os.path.basename(zip_path)
    folder_name = os.path.splitext(zip_filename)[0]
    extract_to = os.path.join(base_dir, folder_name)

    # Create the directory if it doesn't exist
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)

    # Extract the ZIP file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(path=extract_to)

    return extract_to


if __name__ == "__main__":
    zip_file_path = '0.zip' 
    extracted_dir = extract_zip(zip_file_path)
    print(f"Extracted to: {extracted_dir}")

Extracted to: 0


In [5]:
leak_directory = "0"

# Path to the __MACOSX folder
macosx_folder = os.path.join(leak_directory, "__MACOSX")

# Check if __MACOSX exists and remove it - creates issues when analyzing the data, and its not needed, made automatically by MacOS
if os.path.exists(macosx_folder) and os.path.isdir(macosx_folder):
    shutil.rmtree(macosx_folder)
    print(f"Deleted - {macosx_folder}")
else:
    print(f"Folder not found - {macosx_folder}")

# Organize files by extension into subfolders
for root, dirs, files in os.walk(leak_directory):
    for file in files:
        # Skip hidden files and __MACOSX if any reappear
        if file.startswith('.') or '__MACOSX' in root:
            continue

        # Get the file extension (in lowercase, without the dot)
        file_extension = os.path.splitext(file)[1].lower().lstrip('.')
        if not file_extension:
            file_extension = "no_extension"

        # Define the new subfolder path
        subfolder_path = os.path.join(leak_directory, file_extension)

        # Create the subfolder if it doesn't exist
        os.makedirs(subfolder_path, exist_ok=True)

        # Define source and destination paths
        source_path = os.path.join(root, file)
        destination_path = os.path.join(subfolder_path, file)

        # Move the file if source and destination are not the same
        if os.path.abspath(source_path) != os.path.abspath(destination_path):
            shutil.move(source_path, destination_path)

# Remove any empty folders within the parent directory
for dirpath, dirnames, filenames in os.walk(leak_directory, topdown=False):
    if not dirnames and not filenames:
        try:
            os.rmdir(dirpath)
            print(f"Removed empty folder: {dirpath}")
        except OSError:
            pass  

Deleted - 0/__MACOSX


In [6]:
# Delete Duplicate of 0 which is empty

# Directory to delete
EXTRACT_DIR = '0/0'

# Check if the directory exists and delete it
if os.path.isdir(EXTRACT_DIR):
    shutil.rmtree(EXTRACT_DIR)
    print(f"Directory '{EXTRACT_DIR}'deleted")
else:
    print(f"Directory '{EXTRACT_DIR}' does not exist")

Directory '0/0'deleted


# **DATA PARSING**

# Dataframe with all file location

In [7]:
def get_folder_file_dataframe(root_dir):
    folders = []
    for file_name in os.listdir(root_dir):
        full_path = os.path.join(root_dir, file_name)
        if os.path.isdir(full_path):
            folders.append(file_name)

    series_list = []
    for folder in folders:
        folder_path = os.path.join(root_dir, folder)
        files = [
            os.path.join(folder_path, file)
            for file in os.listdir(folder_path)
            if os.path.isfile(os.path.join(folder_path, file)) and not file.startswith('.')
        ]
        s = pd.Series(files, name=folder)
        series_list.append(s)

    df = pd.concat(series_list, axis=1)
    return df

In [8]:
df = get_folder_file_dataframe(leak_directory)
df.head(30)       

,md,png,log,txt
0,0/md/dbc9c90e-a3e6-4d71-bb93-5fb8394095ac.md,0/png/64bba692-d430-440c-9f1e-2575f45770af_6.png,0/log/77010155050.log,0/txt/IDNET.txt
1,0/md/28.md,0/png/12756724-394c-4576-b373-7c53f1abbd94_0.png,0/log/77753527617.log,0/txt/IDTV.txt
2,0/md/5.md,0/png/f179eb06-0c53-44df-a13f-570be23355bb_1.png,0/log/tele2-lbs.log,0/txt/beeline-77774042222.txt
3,0/md/38.md,0/png/5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_24.png,0/log/tele2-cdr.log,0/txt/beeline-77051056626.txt
4,0/md/9d7bc879-3250-4013-ac04-5ff9bd6dff40.md,0/png/5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_30.png,0/log/tele2-crm.log,0/txt/beeline-crm.txt
5,0/md/18.md,0/png/5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_18.png,0/log/77783030133.log,0/txt/UBSCRIBER.txt
6,0/md/9fd06037-11f1-4ad5-9a7d-cbfb3fa4193b.md,0/png/0-adaf869e-920a-4a17-91bd-e2ef3125c10e.png,NaN,0/txt/beeline-cdr.txt
7,0/md/3348953d-66e9-4cac-8675-65bb5f2ef929.md,0/png/5387a301-0af8-4e24-a197-20189f87b9ef_8.png,NaN,0/txt/CRM.txt
8,0/md/1.md,0/png/0-32eb7662-f212-4811-a7c1-1cfeb121cd99.png,NaN,0/txt/LAC.txt
9,0/md/19.md,0/png/912204cb-8ab7-48b8-9abf-d803f3804d08_11.png,NaN,0/txt/beeline-lbs.txt


# Use LLM to classify the source files

In [9]:
import os
import concurrent.futures
import pandas as pd
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from tqdm import tqdm

In [10]:
llm = Ollama(model="llama3.1:8b")

prompt_template = PromptTemplate(
    input_variables=["content"],
    template="""
You are analyzing the content of a file.

File content:
\"\"\"
{content}
\"\"\"

1. Classify the content into one of the following categories ONLY: chats, images, other.
2. State your confidence in the classification as one of: high, medium, or low.
3. I will have to make a csv, please give me a list of headers based on the content. E.g. "[<header_name>, <heaer_name2>, etc.]
Respond in the following format:
Category: <chats|images|other>
Confidence: <high|medium|low>
"""
)

chain = LLMChain(llm=llm, prompt=prompt_template)

# Preprocess the files to the first 20 lines to shorten analysis times
def preprocess_first_20_lines(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            lines = [line.strip() for _, line in zip(range(20), f)]
            content = " ".join(lines)
        return file_path, content
    except Exception:
        return file_path, ""

# Use only the first column of the DataFrame
first_column = df.columns[0]
file_paths = df[first_column].dropna().unique().tolist()

# Preprocess the file for faster classification
with concurrent.futures.ThreadPoolExecutor() as executor:
    file_data = list(executor.map(preprocess_first_20_lines, file_paths))

# Classify and collect by category
valid_categories = {"chats", "images", "other"}
categorized_files = {cat: [] for cat in valid_categories}

for file_path, content in tqdm(file_data, desc="Classifying files"):
    if not content:
        continue

    try:
        response = chain.run(content=content).strip().lower()
        lines = response.splitlines()

        category = ""
        for line in lines:
            if line.startswith("category:"):
                category = line.replace("category:", "").strip()
                break
        if category not in valid_categories:
            category = "other"

        categorized_files[category].append(file_path)

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Convert to DataFrame (columns = categories)
# Determine the maximum list length among all categories
max_len = 0
for file_list in categorized_files.values():
    if len(file_list) > max_len:
        max_len = len(file_list)

# Pad each list with None to match max length
padded = {}
for category, file_list in categorized_files.items():
    padding_needed = max_len - len(file_list)
    padded[category] = file_list + [None] * padding_needed
result_df = pd.DataFrame(padded)
result_df.to_csv("classified_by_category.csv", index=False)
result_df.head()

/tmp/ipykernel_313/23640893.py:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)
Classifying files:   0%|          | 0/70 [00:00<?, ?it/s]/tmp/ipykernel_313/23640893.py:51: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = chain.run(content=content).strip().lower()
Classifying files: 100%|██████████| 70/70 [03:58<00:00,  3.40s/it] 


,other,images,chats
0,0/md/9d7bc879-3250-4013-ac04-5ff9bd6dff40.md,0/md/dbc9c90e-a3e6-4d71-bb93-5fb8394095ac.md,0/md/28.md
1,None,0/md/9fd06037-11f1-4ad5-9a7d-cbfb3fa4193b.md,0/md/5.md
2,None,0/md/3348953d-66e9-4cac-8675-65bb5f2ef929.md,0/md/38.md
3,None,0/md/07f179c5-5705-4dbd-94a7-66eed1e066b0.md,0/md/18.md
4,None,0/md/01cdc26f-e773-4ad7-8808-d04abf16aae7.md,0/md/1.md


In [11]:
import os
import pandas as pd
import re
from collections import defaultdict

# Script to check for cross refernced files

In [12]:
import pandas as pd
import os
import re
from collections import defaultdict

In [13]:
# Extract all unique file paths from the DataFrame 
file_paths = df.stack().dropna().unique().tolist()

# Read file content
def read_file(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            return f.read()
    except:
        return "Failed to open file"

# Read file contents into a dictionary 
file_contents = {path: read_file(path) for path in file_paths}

# Build a reference map (which files mention which) 
reference_map = defaultdict(list)

for source_path, content in file_contents.items():
    if content is None:
        continue  # skip if content is None

    for target_path in file_paths:
        if target_path == source_path:
            continue  # skip comparing file to itself

        target_filename = os.path.basename(target_path)
        pattern = re.escape(target_filename)

        if re.search(rf'\b{pattern}\b', content):
            reference_map[source_path].append(target_path)

# Convert reference map to a DataFrame
ref_df = pd.DataFrame([
    {"source_file": src, "mentions": tgt}
    for src, tgts in reference_map.items()
    for tgt in tgts
])

# Group mentions into lists per source_file
ref_summary = ref_df.groupby("source_file")["mentions"].apply(list).reset_index()

# Some source_file names are also in mentions, therefore they aren't a source file anymore
source_files = set(ref_summary["source_file"])
mentioned_files = set(file for mention_list in ref_summary["mentions"] for file in mention_list)
common_files = source_files.intersection(mentioned_files)

# Filter out common files from the DataFrame
filtered_ref_summary = ref_summary[~ref_summary["source_file"].isin(common_files)].reset_index(drop=True)

filtered_ref_summary.to_csv("file_reference_map.csv", index=False)
print(filtered_ref_summary.head(10))
print("Unique file paths in original DataFrame:", df.stack().nunique())

                                    source_file  \
0  0/md/01cdc26f-e773-4ad7-8808-d04abf16aae7.md   
1                                    0/md/10.md   
2                                    0/md/13.md   
3                                    0/md/15.md   
4  0/md/178e3898-903d-47cf-bfbe-061e7dc18895.md   
5                                    0/md/19.md   
6                                     0/md/2.md   
7                                    0/md/20.md   
8                                    0/md/21.md   
9                                    0/md/22.md   

                                            mentions  
0  [0/png/01cdc26f-e773-4ad7-8808-d04abf16aae7_1_...  
1  [0/png/0-6bcc0131-e4ad-421e-bb1f-d8ebe5eeec7b....  
2  [0/png/0-adaf869e-920a-4a17-91bd-e2ef3125c10e....  
3  [0/png/0-b8cea3b1-4dde-4438-9b1a-6faf690bbad0....  
4  [0/png/178e3898-903d-47cf-bfbe-061e7dc18895_8....  
5  [0/png/6848748d-2881-4c26-b153-fcd5373d2f1c.pn...  
6  [0/md/07f179c5-5705-4dbd-94a7-66eed1e066b0.md,... 

# Check for more linkages in the files + add the missing source files from the chats

In [14]:
import pandas as pd
import os
import re
from ast import literal_eval
import ast

In [15]:
def is_readable_text_file(path):
    return any(path.endswith(ext) for ext in ['txt', 'md', 'rst', 'tex', 'nfo', 'readme', 'rtf', 'doc', 'docx',
    'cfg', 'conf', 'config', 'ini', 'json', 'yaml', 'yml', 'toml',
    'log', 'lst', 'cnf', 'properties', 'prefs',
    'csv', 'tsv', 'dat', 'db', 'dbf', 'sql', 'xml',
    'ssv', 'psv', 'jsonl', 'parquet', 'orc',
    'html', 'htm', 'xhtml', 'xht', 'css', 'js',
    'jsx', 'ts', 'tsx', 'vue', 'erb', 'ejs', 'jsp',
    'liquid', 'handlebars', 'hbs', 'mustache',
    'py', 'pyw', 'ipynb', 'java', 'c', 'cpp', 'h', 'hpp', 'cs',
    'sh', 'bash', 'zsh', 'ksh', 'bat', 'cmd', 'ps1',
    'r', 'jl', 'pl', 'pm', 'rb', 'go', 'lua', 'php',
    'swift', 'scala', 'dart', 'asm', 'groovy',
    'rmd', 'sage', 'nb',
    'env', 'gradle', 'makefile', 'mak', 'mk',
    'dockerfile', 'gitignore', 'gitattributes', 'gitmodules',
    'cmake', 'make', 'ninja', 'build',
    'manifest', 'manifest.json', 'vtt', 'srt', 'resx', 'strings',
    'lang', 'po', 'mo', 'pot', 'msg', 'textbundle',
    'rego', 'tf', 'tfvars', 'cue', 'bzl', 'bazel', 'nix', 'dhall',
    'adoc', 'asciidoc', 'creole', 'mediawiki', 'wiki', 'org',
    'eml', 'msg', 'mbox', 'mail', 'ics', 'vcf'])

mentions2_list = []

# Iterate over each row of the summary
for _, row in filtered_ref_summary.iterrows():
    mention_dict = {}

    # For each mentioned file read file contents and create a dictionary key: location - values: file contents
    for mentioned_file in row["mentions"]:
        if not is_readable_text_file(mentioned_file):
            continue  # Skip unreadable formats

        # Read file content
        content = read_file(mentioned_file)
        if content in [None, "Failed to open file"]:
            continue

        # Search for any file-type references using regex
        # Match anything with a name and known file extension
        found_files = re.findall(r'[\w\-/]*\.\w+', content)

        # Filter duplicates and add to the mention_dict
        if found_files:
            mention_dict[mentioned_file] = list(set(found_files))

    mentions2_list.append(mention_dict)

# Add this as a new column
filtered_ref_summary["mentions2"] = mentions2_list

# Get the list of chat files from result_df - LLM clasification
chat_files = result_df['chats'].dropna().unique()

# Get the list of source files already in filtered_ref_summary
source_files = filtered_ref_summary['source_file'].dropna().unique()

# Find missing chat files and collect chat files that are not found in source files
existing_source_files = set(source_files)
missing_chat_files = []
for chat_file in chat_files:
    if chat_file not in existing_source_files:
        missing_chat_files.append(chat_file)

# Create a new DataFrame with empty mentions and linkage columns
new_rows = pd.DataFrame({
    'source_file': missing_chat_files,
})

# Append the new rows to ref_summary
filtered_ref_summary = pd.concat([filtered_ref_summary, new_rows], ignore_index=True)
        
# Save the updated DataFrame to CSV
filtered_ref_summary.to_csv("file_reference_map_extended.csv", index=False)
print(filtered_ref_summary.columns)

Index(['source_file', 'mentions', 'mentions2'], dtype='object')


# Structuring the Data into CSV - Step 1 all MD files

In [16]:
import os
import pandas as pd
from pathlib import Path
from bs4 import BeautifulSoup

In [17]:
# Create a folder and dumb all csvs
output_dir = "csvs"
os.makedirs(output_dir, exist_ok=True)

# Get all MD files from the DataFrame
all_file_paths = pd.unique(result_df.values.ravel('K'))
md_files = []
for file_path in all_file_paths:
    if isinstance(file_path, str) and file_path.endswith(".md") and os.path.exists(file_path):
        md_files.append(file_path)
        
# Use the LLM's clasification of MD files
chat_files = set(result_df['chats'].dropna().astype(str))

# Extract visible text + ANY file reference 
def extract_text_and_files(td):
    text = td.get_text(strip=True)
    links = []
    for tag in td.find_all(['a', 'img']):
        link = tag.get('href') or tag.get('src')
        if link:
            fname = os.path.basename(link.strip("'\""))
            links.append(fname)
    return text + (" " + " ".join(links) if links else "")

# This method extracts chat table from HTML
def process_chat_md(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
        soup = BeautifulSoup(content, "lxml")
        rows = soup.find_all("tr")[1:]
        data = []
        for row in rows:
            cols = row.find_all("td")
            if len(cols) == 4:
                data.append({
                    "Time": cols[0].get_text(strip=True),
                    "From": cols[1].get_text(strip=True),
                    "To": cols[2].get_text(strip=True),
                    "Message": extract_text_and_files(cols[3])
                })
        return pd.DataFrame(data) if data else None
    except Exception as e:
        print(f"Error processing chat file {file_path}: {e}")
        return None

# Extract the contents of the other file types that the LLM classified
def process_reference_md(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
        soup = BeautifulSoup(content, "lxml")
        matches = []
        for tag in soup.find_all(['a', 'img']):
            link = tag.get('href') or tag.get('src')
            if link:
                fname = os.path.basename(link.strip("'\""))
                if not fname.startswith("0-"):
                    matches.append(fname)
        matches = list(set(matches))
        return pd.DataFrame([{"File_Name": f} for f in matches]) if matches else None
    except Exception as e:
        print(f"Error processing reference file {file_path}: {e}")
        return None

# Loop through all MD filesand call the above methods for the afferent file type
for file_path in md_files:
    csv_name = os.path.basename(file_path).replace(".md", ".csv")
    csv_path = os.path.join(output_dir, csv_name)

    if not os.path.exists(csv_path):
        if file_path in chat_files:
            df = process_chat_md(file_path)
        else:
            df = process_reference_md(file_path)

        if df is not None:
            df.to_csv(csv_path, index=False)
        else:
            print(f"Couldn't save to CSV: {file_path}")

In [18]:
import os
import ast
from pathlib import Path

In [19]:
csv_dir = "csvs"

# Replace MD with CSV if the corresponding file exists
def replace_md_with_csv(item):
    if isinstance(item, str) and item.endswith(".md"):
        base_name = Path(item).stem  # .stem extracts the filename with no extension
        # CSV search
        csv_path = os.path.join(csv_dir, f"{base_name}.csv")
        if os.path.exists(csv_path):
            return csv_path
        else:
            print(f"CSV {item} not found")
    return item

# Recursively handle strings, lists, and dictionaries
def process_column_cell(cell):
    if isinstance(cell, str):
        return replace_md_with_csv(cell)
    
    elif isinstance(cell, list):
        new_list = []
        for item in cell:
            new_item = process_column_cell(item)
            new_list.append(new_item)
        return new_list

    elif isinstance(cell, dict):
        new_dict = {}
        for key, value in cell.items():
            new_key = replace_md_with_csv(key)
            new_value = process_column_cell(value)
            new_dict[new_key] = new_value
        return new_dict

    else:
        return cell
    
for col in filtered_ref_summary.columns:
    filtered_ref_summary[col] = filtered_ref_summary[col].apply(process_column_cell)

filtered_ref_summary.to_csv("filtered_ref_summary_csv_replaced.csv", index=False)
filtered_ref_summary.head()        

,source_file,mentions,mentions2
0,csvs/01cdc26f-e773-4ad7-8808-d04abf16aae7.csv,[0/png/01cdc26f-e773-4ad7-8808-d04abf16aae7_1_...,{}
1,csvs/10.csv,[0/png/0-6bcc0131-e4ad-421e-bb1f-d8ebe5eeec7b....,{'csvs/12756724-394c-4576-b373-7c53f1abbd94.cs...
2,csvs/13.csv,[0/png/0-adaf869e-920a-4a17-91bd-e2ef3125c10e....,{'csvs/585875ff-f8c5-4a02-acd7-fef37dc9ff11.cs...
3,csvs/15.csv,[0/png/0-b8cea3b1-4dde-4438-9b1a-6faf690bbad0....,{}
4,csvs/178e3898-903d-47cf-bfbe-061e7dc18895.csv,[0/png/178e3898-903d-47cf-bfbe-061e7dc18895_8....,{}


# Structuring the Data into CSV - Step 2 OCR on image files

LIST OF FILES FOR TEST: 5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_0.png, 08a6bcd3-6477-4252-8f35-4f8f80d114f9.png, 1afcf93d-50f1-4f1e-896d-87b0da7519f7.png, 32eb7662-f212-4811-a7c1-1cfeb121cd99.png, 6bcc0131-e4ad-421e-bb1f-d8ebe5eeec7b.png, 79d9b7f2-cfe4-4615-9b75-8fea33fc0c9d.png, 4ae9bf34-c16c-4684-aa92-fec65a151275.png, 5a84cde3-7175-4044-8c88-d4c883a8fd38.png, e705d192-90ee-4fd1-9dcd-061958d1817f.png, 912204cb-8ab7-48b8-9abf-d803f3804d08_4.png

# **TEST-1: Extraction with LLAVA**

In [20]:
import base64
import requests
import os

# List of image file paths
image_paths = [
    "0/png/5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_0.png", 
    "0/png/08a6bcd3-6477-4252-8f35-4f8f80d114f9.png", 
    "0/png/1afcf93d-50f1-4f1e-896d-87b0da7519f7.png", 
    "0/png/32eb7662-f212-4811-a7c1-1cfeb121cd99.png", 
    "0/png/6bcc0131-e4ad-421e-bb1f-d8ebe5eeec7b.png", 
    "0/png/79d9b7f2-cfe4-4615-9b75-8fea33fc0c9d.png", 
    "0/png/4ae9bf34-c16c-4684-aa92-fec65a151275.png", 
    "0/png/5a84cde3-7175-4044-8c88-d4c883a8fd38.png", 
    "0/png/e705d192-90ee-4fd1-9dcd-061958d1817f.png", 
    "0/png/912204cb-8ab7-48b8-9abf-d803f3804d08_4.png"
]

# Define the prompt
prompt = "EXTRACT AND ONLY GIVE ME THE Chinese text in the given image. NO TRANSLATION, ONLY extract the TEXT IN CHINESE MANDARIN FROM THE PNG FILE"

# Process each image
for image_path in image_paths:
    with open(image_path, "rb") as f:
        image_b64 = base64.b64encode(f.read()).decode("utf-8")

    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "llava:13b",
            "prompt": prompt,
            "images": [image_b64],
            "stream": False
        },
    )

    filename = os.path.basename(image_path)
    if response.ok:
        output_text = response.json()["response"]
    else:
        output_text = f"Error: {response.text}"

    print(f"{filename}:\n{output_text.strip()}\n")

5a6b122c-39c1-4581-8c1f-2d6f36a9f8a0_0.png:
The Chinese text on the image is: "未来医疗技术与科学研究开展方向".

08a6bcd3-6477-4252-8f35-4f8f80d114f9.png:
I'm unable to process images or provide translations directly from this interface. If you need assistance with the Chinese text in the image, please type out the text in a separate message, and I will be happy to help you with its translation or any other question you may have!

1afcf93d-50f1-4f1e-896d-87b0da7519f7.png:
你好！我有一个问题想要请教。我现在正在做一个简单的项目，需要购买一些电子设备。如果可以的话，能否向您请教一下价格和采购建议？谢谢！

32eb7662-f212-4811-a7c1-1cfeb121cd99.png:
The image you've provided shows a screenshot of a computer interface with several lines of Chinese text. Here is the extracted Chinese text from the visible parts of the screen:

```
发送人：陈某某
发送时间：2014年6月8日 下午9点37分
标题：关于一些信息的提供
内容：

这是一个测试文本。请问我应该在哪里写？
```

Please note that the text may be incomplete as it only covers part of what is visible in the image.

6bcc0131-e4ad-421e-bb1f-d8ebe5eeec7b.png:
抱歉，由于此图像的质量不够高，我无法识别并提取中文。请提供

# **DATA TRANSLATION Verification - 3 LLMs**

In [21]:
import os
import pandas as pd

In [22]:
# Use the csvs folder and the LLM classification of MD files to create a new df that stores the csvs into categories.

csv_folder = "csvs"

csv_mapping = {}
# Loop through each file in the CSV folder
for filename in os.listdir(csv_folder):
    # Check for csv
    if filename.endswith('.csv'):
        # Replace the ".csv" extension with ".md" to create the key
        md_filename = filename.replace('.csv', '.md')
        
        # Set the value to the full path to the CSV file 
        csv_full_path = os.path.join(csv_folder, filename)
        
        # Add the key-value pair to the mapping
        csv_mapping[md_filename] = csv_full_path

# Step 2: Define function to replace .md path with .csv path
def replace_md_with_csv(val):
    if isinstance(val, str):
        base = os.path.basename(val)
        return csv_mapping.get(base, val)
    return val

df_csvs_llm = result_df.applymap(replace_md_with_csv)
df_csvs_llm.to_csv("updated_with_csv_paths.csv", index=False)
print(df_csvs_llm.head())

                                           other  \
0  csvs/9d7bc879-3250-4013-ac04-5ff9bd6dff40.csv   
1                                           None   
2                                           None   
3                                           None   
4                                           None   

                                          images        chats  
0  csvs/dbc9c90e-a3e6-4d71-bb93-5fb8394095ac.csv  csvs/28.csv  
1  csvs/9fd06037-11f1-4ad5-9a7d-cbfb3fa4193b.csv   csvs/5.csv  
2  csvs/3348953d-66e9-4cac-8675-65bb5f2ef929.csv  csvs/38.csv  
3  csvs/07f179c5-5705-4dbd-94a7-66eed1e066b0.csv  csvs/18.csv  
4  csvs/01cdc26f-e773-4ad7-8808-d04abf16aae7.csv   csvs/1.csv  


# **TEST-1: gemma3:27b**

In [79]:
import pandas as pd
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

csv_path = 'csvs/37.csv'
output_path = '3llms_10.csv'  # Output in project root directory
df = pd.read_csv(csv_path)

def get_context(df, index, window=5):
    start = max(0, index - window)
    end = min(len(df), index + window + 1)
    return "\n".join(df['Message'].iloc[start:end].dropna())

def query_ollama(prompt, model="gemma3:27b"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload, timeout=90)
        response.raise_for_status()
        return response.json()["response"].strip()
    except Exception as e:
        print(f"Ollama error: {e}")
        return "[Translation Error]"

def translate_row(index):
    target = df.at[index, 'Message']
    if pd.isna(target):
        return index, ""
    context = get_context(df, index)
    prompt = (
        "You are translating Chinese messages to English. Below is a series of related messages. "
        "Use the full context to understand the meaning, but only translate the specific message provided.\n\n"
        f"Context:\n{context}\n\n"
        f"Message to translate:\n{target}\n\n"
        "ONLY RETURN the English translation of the message, ANYTHING ELSE IS FORBIDDEN! "
        "If you encounter a filename or file reference, preserve it exactly as it appears, DON'T add anything!"
    )
    translation = query_ollama(prompt)
    return index, translation

translations = [""] * len(df)

futures = []
with ThreadPoolExecutor(max_workers=8) as executor:
    for index in df.index:
        futures.append(executor.submit(translate_row, index))

    for future in tqdm(as_completed(futures), total=len(futures), desc="Translating with Ollama"):
        idx, result = future.result()
        translations[idx] = result

translated_df = pd.DataFrame({'gemma3:27b': translations})
translated_df.to_csv(output_path, index=False)

Translating with Ollama:   7%|▋         | 22/329 [02:06<29:27,  5.76s/it] 


KeyboardInterrupt: 

Ollama error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=90)
Ollama error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=90)
Ollama error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=90)
Ollama error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=90)
Ollama error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=90)Ollama error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=90)
Ollama error: HTTPConnectionPool(host='localhost', port=11434): Read timed out. (read timeout=90)

# **TEST-2: 7shi/llama-translate:8b-q4_K_M**

In [10]:
import pandas as pd
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

source_csv_path = 'csvs/10.csv'
existing_translation_path = '3llms_10.csv'
new_model_name = '7shi/llama-translate:8b-q4_K_M'

# Load original messages and previously translated file
source_df = pd.read_csv(source_csv_path)
translated_df = pd.read_csv(existing_translation_path)

def get_context(df, index, window=5):
    start = max(0, index - window)
    end = min(len(df), index + window + 1)
    return "\n".join(df['Message'].iloc[start:end].dropna())

def query_ollama(prompt, model):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload, timeout=90)
        response.raise_for_status()
        return response.json()["response"].strip()
    except Exception as e:
        print(f"Ollama error: {e}")
        return "[Translation Error]"

def translate_row(index):
    target = source_df.at[index, 'Message']
    if pd.isna(target):
        return index, ""
    context = get_context(source_df, index)
    prompt = (
        "Transalte Mandarin to English. Below is a series of related messages. "
        f"Context:\n{context}\n\n"
        f"Message to translate:\n{target}\n\n"
    )
    translation = query_ollama(prompt, model=new_model_name)
    return index, translation

# Prepare for parallel processing
translations = [""] * len(source_df)
futures = []
with ThreadPoolExecutor(max_workers=8) as executor:
    for index in source_df.index:
        futures.append(executor.submit(translate_row, index))

    for future in tqdm(as_completed(futures), total=len(futures), desc=f"Translating with {new_model_name}"):
        idx, result = future.result()
        translations[idx] = result

translated_df[new_model_name] = translations
translated_df.to_csv(existing_translation_path, index=False)

Translating with 7shi/llama-translate:8b-q4_K_M: 100%|██████████| 149/149 [08:52<00:00,  3.57s/it]


# **TEST-3: yi:34b**

In [11]:
import pandas as pd
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# File paths
source_csv_path = 'csvs/10.csv'
existing_translation_path = '3llms_10.csv'
new_model_name = 'yi:34b'

# Load original source messages and existing translations
source_df = pd.read_csv(source_csv_path)
translated_df = pd.read_csv(existing_translation_path)

# Context window function
def get_context(df, index, window=5):
    start = max(0, index - window)
    end = min(len(df), index + window + 1)
    return "\n".join(df['Message'].iloc[start:end].dropna())

def query_ollama(prompt, model):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload, timeout=90)
        response.raise_for_status()
        return response.json()["response"].strip()
    except Exception as e:
        print(f"Ollama error: {e}")
        return "[Translation Error]"

# Translate a single row
def translate_row(index):
    target = source_df.at[index, 'Message']
    if pd.isna(target):
        return index, ""
    context = get_context(source_df, index)
    prompt = (
        "You are translating Chinese messages to English. Below is a series of related messages. "
        "Use the full context to understand the meaning, but only translate the specific message provided.\n\n"
        f"Context:\n{context}\n\n"
        f"Message to translate:\n{target}\n\n"
        "ONLY RETURN the English translation of the message, ANYTHING ELSE IS FORBIDDEN! "
        "If you encounter a filename or file reference, preserve it exactly as it appears, DON'T add anything!"
    )
    translation = query_ollama(prompt, model=new_model_name)
    return index, translation

# Run translations in parallel
translations = [""] * len(source_df)
futures = []
with ThreadPoolExecutor(max_workers=8) as executor:
    for index in source_df.index:
        futures.append(executor.submit(translate_row, index))

    for future in tqdm(as_completed(futures), total=len(futures), desc=f"Translating with {new_model_name}"):
        idx, result = future.result()
        translations[idx] = result

translated_df[new_model_name] = translations
translated_df.to_csv(existing_translation_path, index=False)

Translating with yi:34b: 100%|██████████| 149/149 [07:20<00:00,  2.96s/it]


# **Levenshtein Distance Between the 3 Translations**

In [14]:
import pandas as pd
import Levenshtein
from itertools import combinations

csv_path = '3llms_10.csv'
df = pd.read_csv(csv_path)

# Identify all translation columns 
translation_columns = [col for col in df.columns if not col.startswith('Levenshtein')]

# Compute Levenshtein distance for all possible combinations
for col1, col2 in combinations(translation_columns, 2):
    new_col_name = f"Levenshtein ({col1} vs {col2})"
    df[new_col_name] = [
        Levenshtein.distance(str(a), str(b)) if pd.notna(a) and pd.notna(b) else None
        for a, b in zip(df[col1], df[col2])
    ]

# Save updated DataFrame back to the same CSV
df.to_csv(csv_path, index=False)

# **CHATS TRANSLATION - GEMMA3:27B**

In [23]:
import pandas as pd
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Chats - CSV loop
for csv_path in df_csvs_llm['chats']:
    print(f"Processing: {csv_path}")
    
    # Load the individual CSV file
    df = pd.read_csv(csv_path)
    df['message_translation'] = ""

    # Function to get context around the target message
    def get_context(df, index, window=5):
        start = max(0, index - window)
        end = min(len(df), index + window + 1)
        return "\n".join(df['Message'].iloc[start:end].dropna())

    def query_ollama(prompt, model="gemma3:27b"):
        url = "http://localhost:11434/api/generate"
        payload = {
            "model": model,
            "prompt": prompt,
            "stream": False
        }
        try:
            response = requests.post(url, json=payload, timeout=90)
            response.raise_for_status()
            return response.json()["response"].strip()
        except Exception as e:
            print(f"Ollama error: {e}")
            return "[Translation Error]"

    # Function to process one row
    def translate_row(index):
        target = df.at[index, 'Message']
        if pd.isna(target):
            return index, ""
        context = get_context(df, index)
        prompt = (
            "You are translating Chinese messages to English. Below is a series of related messages. "
            "Use the full context to understand the meaning, but only translate the specific message provided.\n\n"
            f"Context:\n{context}\n\n"
            f"Message to translate:\n{target}\n\n"
            "ONLY RETURN the English translation of the message, ANYTHING ELSE IS FORBIDDEN! "
            "If you encounter a filename or file reference, preserve it exactly as it appears, DON'T add anything!"
        )
        translation = query_ollama(prompt)
        return index, translation

    # Run translations in parallel
    futures = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        for index in df.index:
            futures.append(executor.submit(translate_row, index))

        for future in tqdm(as_completed(futures), total=len(futures), desc=f"Translating: {csv_path}"):
            idx, result = future.result()
            df.at[idx, 'message_translation'] = result

    # Save updated DataFrame back to the original file
    df.to_csv(csv_path, index=False)
    print(f"Finished and saved: {csv_path}\n")

Processing: csvs/28.csv


Translating: csvs/28.csv: 100%|██████████| 46/46 [02:55<00:00,  3.81s/it]


Finished and saved: csvs/28.csv

Processing: csvs/5.csv


Translating: csvs/5.csv: 100%|██████████| 17/17 [00:24<00:00,  1.46s/it]


Finished and saved: csvs/5.csv

Processing: csvs/38.csv


...

Finished and saved: csvs/7.csv

Processing: csvs/6.csv


Translating: csvs/6.csv: 100%|██████████| 136/136 [03:08<00:00,  1.38s/it]


Finished and saved: csvs/6.csv

Processing: csvs/2.csv


Translating: csvs/2.csv: 100%|██████████| 500/500 [16:03<00:00,  1.93s/it]

Finished and saved: csvs/2.csv



In [5]:
import pandas as pd

# Load the original CSV
df = pd.read_csv("csvs/10.csv")

# Print all column names to verify exact labels
print("CSV Columns:")
print(df.columns.tolist())

# Drop the specified columns (only if they exist)
columns_to_drop = ["message_translation", "financial_detection"]
df_cleaned = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Overwrite the original file
df_cleaned.to_csv("csvs/10.csv", index=False)  # Overwriting the same file

CSV Columns:
['Time', 'From', 'To', 'Message']


# **OCR TRANSLATION - GEMMA3:27b**

In [7]:
import os
import pandas as pd
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

ocr_folder_path = 'ocr_to_csv/'

def query_ollama(prompt, model="gemma3:27b"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload, timeout=90)
        response.raise_for_status()
        return response.json()["response"].strip()
    except Exception as e:
        print(f"Ollama error: {e}")
        return "[Translation Error]"

def translate_row(index, row_text):
    if pd.isna(row_text) or not row_text.strip():
        return index, ""
    prompt = (
        "You are translating Chinese text into fluent English. Translate the following message:\n\n"
        f"{row_text}\n\n"
        "ONLY RETURN the English translation. DO NOT add commentary or extra formatting."
    )
    return index, query_ollama(prompt)

ocr_files = sorted([f for f in os.listdir(ocr_folder_path) if f.endswith('.csv')])

for file_idx, file in enumerate(tqdm(ocr_files, desc="Translating OCR Files", unit="file")):
    print(f"\nProcessing file {file_idx + 1} of {len(ocr_files)}: {file}")
    path = os.path.join(ocr_folder_path, file)
    df = pd.read_csv(path)

    if 'ocr extraction' not in df.columns:
        print(f" Skipping {file} (missing 'ocr extraction' column)")
        continue

    df['ocr translation'] = ""

    futures = []
    with ThreadPoolExecutor(max_workers=8) as executor:
        for idx, row in df.iterrows():
            futures.append(executor.submit(translate_row, idx, row['ocr extraction']))

        for future in as_completed(futures):
            idx, translated = future.result()
            df.at[idx, 'ocr translation'] = translated

    df.to_csv(path, index=False)
    print(f" Saved translated file: {file}")

Translating OCR Files:   0%|          | 0/490 [00:00<?, ?file/s]


Processing file 1 of 490: 0-08a6bcd3-6477-4252-8f35-4f8f80d114f9.csv


Translating OCR Files:   0%|          | 1/490 [01:08<9:17:52, 68.45s/file]

 Saved translated file: 0-08a6bcd3-6477-4252-8f35-4f8f80d114f9.csv

Processing file 2 of 490: 0-0b54af64-c2cd-4acb-9864-73a584aa6ebc.csv


Translating OCR Files:   0%|          | 2/490 [01:24<5:06:57, 37.74s/file]

...

 Saved translated file: fe245192-1f9c-4f28-9b32-046fb7ce7e1e_7.csv

Processing file 489 of 490: fe245192-1f9c-4f28-9b32-046fb7ce7e1e_8.csv


Translating OCR Files: 100%|█████████▉| 489/490 [4:24:45<00:38, 38.50s/file]

 Saved translated file: fe245192-1f9c-4f28-9b32-046fb7ce7e1e_8.csv

Processing file 490 of 490: fe245192-1f9c-4f28-9b32-046fb7ce7e1e_9.csv


Translating OCR Files: 100%|██████████| 490/490 [4:25:20<00:00, 32.49s/file]

 Saved translated file: fe245192-1f9c-4f28-9b32-046fb7ce7e1e_9.csv


# **Data has been added from MacOS OCR**

# *Changing the files in the chats with the extracted output* 

In [ ]:
import pandas as pd
import os
import re
import ast

In [10]:
mapping_csv_path = 'filtered_ref_summary_csv_replaced.csv'
chat_reference_csv_path = 'updated_with_csv_paths.csv'  # The one shown in the screenshot
ocr_folder_path = 'ocr_to_csv'

# Use the csv file stroing all the paths to filtered chats by the LLM
chat_paths_df = pd.read_csv(chat_reference_csv_path)

mapping_df = pd.read_csv(mapping_csv_path)

# Link md files and the pngs inside based on uuid
md_uuid_to_pngs = {}

for index, row in mapping_df.iterrows():
    mentions2_text = row.get('mentions2', '{}')
    try:
        mentions2 = ast.literal_eval(mentions2_text)
        for md_path, png_list in mentions2.items():
            md_uuid = md_path.split('/')[-1].replace('.csv', '')
            png_names = [os.path.basename(p) for p in png_list]
            md_uuid_to_pngs[md_uuid] = png_names
    except:
        continue

# Function to fetch OCR translation from OCR folder
def get_ocr_translation(png_filename):
    csv_filename = png_filename.replace('.png', '.csv')
    csv_path = os.path.join(ocr_folder_path, csv_filename)

    if os.path.exists(csv_path):
        try:
            df = pd.read_csv(csv_path)
            if 'ocr translation' in df.columns:
                non_empty_translations = df['ocr translation'].dropna().astype(str)
                for text in non_empty_translations:
                    if text.strip():
                        return text
        except Exception as e:
            print(f"Error reading {csv_path}: {e}")
            return ''
    return ''

# Function to replace .png/.md in message with OCR translation
def replace_with_ocr_translation(message):
    if not isinstance(message, str):
        return message

    png_files = re.findall(r'(\d+-[a-f0-9\-]+(?:_[0-9]+)?\.png)', message)
    for png in png_files:
        ocr_text = get_ocr_translation(png)
        if ocr_text:
            message = message.replace(png, f"OCR translation:\n{ocr_text}")

    md_uuids = re.findall(r'([a-f0-9\-]{36})\.md', message)
    for md_uuid in md_uuids:
        png_list = md_uuid_to_pngs.get(md_uuid, [])
        ocr_texts = []
        for png in png_list:
            ocr_text = get_ocr_translation(png)
            if ocr_text:
                ocr_texts.append(f"OCR for {png}:\n{ocr_text}")
        if ocr_texts:
            combined = "\n\n".join(ocr_texts)
            full_md_match = re.search(r'\S*' + re.escape(md_uuid) + r'\.md', message)
            if full_md_match:
                message = message.replace(full_md_match.group(0), combined)

    return message

# Loop over each chat CSV listed in the 'chats' column
for idx, row in chat_paths_df.iterrows():
    chat_csv_path = row['chats']
    if not isinstance(chat_csv_path, str) or not os.path.exists(chat_csv_path):
        print(f"Skipping missing or invalid path: {chat_csv_path}")
        continue

    print(f"Processing: {chat_csv_path}")
    try:
        chat_df = pd.read_csv(chat_csv_path)
        if 'message_translation' in chat_df.columns:
            chat_df['message_translation'] = chat_df['message_translation'].apply(replace_with_ocr_translation)
            chat_df.to_csv(chat_csv_path, index=False)
        else:
            print(f"Column 'message_translation' not found in {chat_csv_path}")
    except Exception as e:
        print(f"Error processing {chat_csv_path}: {e}")

Processing: csvs/28.csv
Processing: csvs/5.csv
Processing: csvs/38.csv
Processing: csvs/18.csv
Processing: csvs/1.csv
Processing: csvs/19.csv
Processing: csvs/29.csv
Processing: csvs/4.csv
Processing: csvs/39.csv
Processing: csvs/16.csv
Processing: csvs/22.csv
Processing: csvs/32.csv
Processing: csvs/26.csv
Processing: csvs/12.csv
Processing: csvs/36.csv
Processing: csvs/27.csv
Processing: csvs/13.csv
Processing: csvs/37.csv
Processing: csvs/17.csv
Processing: csvs/23.csv
Processing: csvs/33.csv
Processing: csvs/24.csv
Processing: csvs/41.csv
Processing: csvs/10.csv
Processing: csvs/9.csv
Processing: csvs/34.csv
Processing: csvs/14.csv
Processing: csvs/20.csv
Processing: csvs/30.csv
Processing: csvs/15.csv
Processing: csvs/21.csv
Processing: csvs/31.csv
Processing: csvs/40.csv
Processing: csvs/11.csv
Processing: csvs/35.csv
Processing: csvs/3.csv
Processing: csvs/7.csv
Processing: csvs/6.csv
Processing: csvs/2.csv


# **TXT AND LOG FILES PARSING**

In [1]:
import csv

# File paths
input_file = '0/txt/CRM.txt'
output_file = 'CRM.csv'

# Read the tab-delimited TXT file and write to a CSV
with open(input_file, 'r', encoding='utf-8') as infile, \
     open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    
    reader = csv.reader(infile, delimiter='\t')
    writer = csv.writer(outfile)

    for row in reader:
        # Skip entirely empty rows
        if any(field.strip() for field in row):
            writer.writerow(row)

print("CSV file 'CRM.csv' created successfully with headers preserved.")

CSV file 'CRM.csv' created successfully with headers preserved.


In [2]:
import os
import csv

# ✅ Define your input and output folders here
SOURCE_FOLDER_1 = "0/log"
SOURCE_FOLDER_2 = "0/txt"
OUTPUT_FOLDER = "logtxt_to_csv"

def detect_format(lines):
    if len(lines) >= 2 and all(c == '-' or c == ' ' for c in lines[1]):
        return 'fixed-width'
    elif ',' in lines[0]:
        return 'csv'
    elif '\t' in lines[0]:
        return 'tsv'
    return 'unknown'

def get_fixed_positions(divider_line):
    positions = []
    in_field = False
    for i, char in enumerate(divider_line):
        if char == '-' and not in_field:
            start = i
            in_field = True
        elif char != '-' and in_field:
            end = i
            positions.append((start, end))
            in_field = False
    if in_field:
        positions.append((start, len(divider_line)))
    return positions

def process_fixed_width(lines, output_path):
    header_line = lines[0]
    divider_line = lines[1]
    data_lines = lines[2:]
    positions = get_fixed_positions(divider_line)
    headers = [header_line[start:end].strip() for start, end in positions]

    with open(output_path, 'w', newline='', encoding='utf-8') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(headers)
        for line in data_lines:
            row = [line[start:end].strip() for start, end in positions]
            if any(row):
                writer.writerow(row)

def process_delimited(lines, output_path, delimiter):
    with open(output_path, 'w', newline='', encoding='utf-8') as out_file:
        writer = csv.writer(out_file)
        for line in lines:
            row = [field.strip().replace(',', '.') if delimiter == '\t' and field.replace(',', '').replace('.', '').isdigit() else field.strip() for field in line.strip().split(delimiter)]
            if any(row):
                writer.writerow(row)

def convert_all_files(source_folders, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for input_dir in source_folders:
        for file in os.listdir(input_dir):
            if not file.lower().endswith(('.txt', '.log')):
                continue

            file_path = os.path.join(input_dir, file)
            base_name = os.path.basename(file)
            output_path = os.path.join(output_folder, os.path.splitext(base_name)[0] + '.csv')

            with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
                lines = f.readlines()

            if not lines:
                continue

            fmt = detect_format(lines)

            try:
                if fmt == 'fixed-width':
                    process_fixed_width(lines, output_path)
                elif fmt == 'tsv':
                    process_delimited(lines, output_path, delimiter='\t')
                elif fmt == 'csv':
                    process_delimited(lines, output_path, delimiter=',')
                else:
                    print(f"Skipped unknown format: {file}")
                    continue

                print(f"Converted: {file} → {os.path.basename(output_path)}")
            except Exception as e:
                print(f"Failed to convert {file}: {e}")

if __name__ == "__main__":
    convert_all_files([SOURCE_FOLDER_1, SOURCE_FOLDER_2], OUTPUT_FOLDER)

Skipped unknown format: 77010155050.log
Skipped unknown format: 77753527617.log
Converted: tele2-lbs.log → tele2-lbs.csv
Converted: tele2-cdr.log → tele2-cdr.csv
Converted: tele2-crm.log → tele2-crm.csv
Skipped unknown format: 77783030133.log
Converted: IDNET.txt → IDNET.csv
Converted: IDTV.txt → IDTV.csv
Converted: beeline-77774042222.txt → beeline-77774042222.csv
Converted: beeline-77051056626.txt → beeline-77051056626.csv
Converted: beeline-crm.txt → beeline-crm.csv
Skipped unknown format: UBSCRIBER.txt
Converted: beeline-cdr.txt → beeline-cdr.csv
Converted: CRM.txt → CRM.csv
Converted: LAC.txt → LAC.csv
Converted: beeline-lbs.txt → beeline-lbs.csv
Converted: Φ»¥σìò.txt → Φ»¥σìò.csv


# **USER ANALYSIS**

In [4]:
import pandas as pd
import os
from collections import Counter

In [8]:
df_csvs = pd.read_csv("updated_with_csv_paths.csv")

def count_messages_per_user_from_csv(csv_path):
    try:
        df = pd.read_csv(csv_path)
    except Exception as e:
        print(f"Error reading {csv_path}: {e}")
        return Counter()

    df.columns = [col.lower() for col in df.columns]

    # Identify the 'from' column
    from_columns = [col for col in df.columns if col in ['from', 'sender', 'user']]
    if not from_columns:
        return Counter()

    from_col = from_columns[0]

    # Drop missing senders and count messages
    user_counts = df[from_col].dropna().astype(str).str.strip()
    return Counter(user_counts[user_counts != ''])

all_message_counts = Counter()

# Loop through each CSV path in the 'chats' column
for csv_path in df_csvs['chats'].dropna().unique():
    if isinstance(csv_path, str) and csv_path.endswith('.csv') and os.path.exists(csv_path):
        counts = count_messages_per_user_from_csv(csv_path)
        all_message_counts.update(counts)

sorted_user_message_counts = sorted(all_message_counts.items(), key=lambda x: x[1], reverse=True)

print("Messages sent per user across all chat logs:")
for user, count in sorted_user_message_counts:
    print(f"- {user}: {count}")

Messages sent per user across all chat logs:
- lengmo: 4981
- Shutd0wn: 3675
- wxid_5390224027312: 1409
- wxid_7p054rmzkhqf21: 894
- wxid_wh6x59w70y3r22: 620
- gzp1991101: 603
- wxid_zb45i0rc71yk21: 590
- wxid_70w3p1jin84k22: 523
- wxid_hlmnhsq64tt722: 483
- adpw90: 235
- qq78263462: 225
- wxid_12n748um1thl21: 192
- wxid_mgh25nentc4u22: 190
- just910420: 154
- wxid_c9yv0nsla3yn22: 116
- nullroot: 103
- wxid_icges6alg8cl21: 91
- wei592628: 87
- wxid_soekgggwnfgm21: 79
- wxid_nv9bv435fz3722: 77
- wxid_xusilpfkh31g21: 61
- ken73224: 58
- hack05112: 45
- tianyi-0608: 45
- wxid_kbys0kvzj4ta12: 38
- SWEET5683yao: 35
- wxid_zbytkn4qjl3r22: 29
- wxid_blw54o1q0q5w22: 24
- yanzi542766277: 24
- dujijiyiqxx: 17
- wxid_jcnxegjccqi441: 11
- wangchao953541: 9
- ibabaimama: 5
- zhangxiaoyan0422: 5
- snipersk: 5
- wxid_q7vkst94g5u011: 5


In [ ]:
import pandas as pd
import os
import re
import requests
from collections import defaultdict, Counter

In [77]:
def query_ollama(prompt, model="gemma3:27b"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    try:
        response = requests.post(url, json=payload, timeout=90)
        response.raise_for_status()
        return response.json()["response"].strip().lower()
    except Exception as e:
        return f"[error: {e}]"

# Prompt builder
def build_prompt(sender, receiver, message_text, honorific):
    return (
        f"There is a conversation between two users:\n"
        f"User A: {sender}\n"
        f"User B: {receiver}\n\n"
        f"The message is as follows:\n"
        f"User A → User B: {message_text}\n\n"
        f"Question: In the sentence '{message_text}', who is {honorific} — "
        f"the sender, the receiver, or a third party?\n\n"
        f"Output format: {honorific} - give the username of the person or "
        f"if they are a thirdparty, no explanation needed"
    )

# Define patterns to detect honorifics 
honorific_keywords = [r'Mr\.', r'Ms\.', r'Mrs\.', r'Miss', r'Brother', r'Sister']

# Pattern to match names 
honorific_pattern = re.compile(
    r'\b(?:' + '|'.join(honorific_keywords) + r')\s+(?:[A-Z]\b|[A-Z][a-z]+)\b',
    re.IGNORECASE
)

# Pattern to check if a message directly addresses someone
direct_address_pattern = re.compile(
    r'^(hi|hello|dear)?\s*(?:' + '|'.join(honorific_keywords) + r')\s+(?:[A-Z]\b|[A-Z][a-z]+)\b[\s,:]',
    re.IGNORECASE
)

# Keywords indicating an indirect mention
indirect_words = {'from', 'of', 'said', 'mentioned', 'told', 'spoke', 'called'}

# Load CSV file containing paths to chat logs
df_paths = pd.read_csv("updated_with_csv_paths.csv")

# Dictionaries to hold results
title_counts = defaultdict(Counter)
title_examples = defaultdict(list)

# Loop through each chat log file
for chat_file in df_paths['chats'].dropna().unique():
    if not (isinstance(chat_file, str) and chat_file.endswith('.csv') and os.path.exists(chat_file)):
        continue

    try:
        df_chat = pd.read_csv(chat_file)
    except Exception as e:
        print(f"Error reading {chat_file}: {e}")
        continue

    # Standardize column names
    df_chat.columns = [col.lower() for col in df_chat.columns]
    message_columns = [col for col in df_chat.columns if 'message_translation' in col]
    sender_column = df_chat.columns[1]
    receiver_column = df_chat.columns[2]

    # Iterate through each message
    for index, row in df_chat.iterrows():
        for message_col in message_columns:
            if pd.isna(row.get(message_col)):
                continue

            message = str(row[message_col]).strip()
            sender = row.get(sender_column)
            receiver = row.get(receiver_column)

            if not receiver or receiver == sender:
                continue

            matches = honorific_pattern.findall(message)
            if len(matches) != 1:
                continue  # Skip ambiguous lines

            honorific = matches[0].strip()

            # Basic direct-address check
            if not re.match(rf"^(hi|hello|dear)?\s*{re.escape(honorific)}[\s,:]", message, re.IGNORECASE):
                continue

            # Use LLM to validate honorific use
            prompt = build_prompt(sender, receiver, message, honorific)
            result = query_ollama(prompt)

            if "thirdparty" in result:
                continue

            # Store valid matches
            title_counts[receiver][honorific] += 1
            title_examples[receiver].append((honorific, os.path.basename(chat_file), index, message))

# Build the summary table
summary_rows = []
for user_id, counter in title_counts.items():
    if not counter:
        continue

    top_honorific, count = counter.most_common(1)[0]

    # Get up to 3 example messages
    examples = [
        f"{file}:{line} - \"{msg}\""
        for t, file, line, msg in title_examples[user_id] if t == top_honorific
    ][:3]

    summary_rows.append({
        "username": user_id,
        "real_name": top_honorific,
        "count": count,
        "examples": " | ".join(examples)
    })

# Convert to DataFrame
df_summary = pd.DataFrame(summary_rows, columns=["username", "real_name", "count", "examples"])

# Print the result
print(df_summary)

# Save to CSV
df_summary.to_csv("honorific_usernames_cleaned.csv", index=False)

              username     real_name  count  \
0             Shutd0wn        Mr. Wu      1   
1               adpw90     Mr. Zheng      1   
2   wxid_5390224027312      Mr. Wang      2   
3  wxid_70w3p1jin84k22   Sister Qian      1   
4         SWEET5683yao  Brother Qing      2   
5               lengmo         Mr. C      4   
6  wxid_7p054rmzkhqf21        Mr. Lu      2   
7            wei592628       Mr. Wei      4   
8             nullroot      Mr. Zhou      2   
9           gzp1991101      Mr. Gong      7   

                                            examples  
0  28.csv:0 - "Mr. Wu, are you available now? I'd...  
1  4.csv:21 - "Mr. Zheng, help me see where this ...  
2  39.csv:311 - "Mr. Wang, there's nothing I can ...  
3  16.csv:104 - "Sister Qian, don't settle for le...  
4  22.csv:6 - "Brother Qing, are you interested?"...  
5  26.csv:1 - "Mr. C, this is a summary of the sa...  
6  24.csv:27 - "Mr. Lu, regarding what Ying menti...  
7  10.csv:1 - "Mr. Wei, roughly what aspec

# *RegEx for IPs and domain names*

In [42]:
import pandas as pd
import os
import re

df_csvs = pd.read_csv("updated_with_csv_paths.csv")

email_pattern = re.compile(r'\b\S+@\S+\.\S+\b')
ip_pattern = re.compile(r'\b(?:\d{1,3}\.){3}\d{1,3}(?::\d{1,5})?\b')
domain_pattern = re.compile(r'\b(?:[a-zA-Z0-9-]+\.)+(?:[a-zA-Z]{2,})\b')
isoon_pattern = re.compile(r'\b(i[-_ ]?soon|auxun|安洵)\b', re.IGNORECASE)

bad_extensions = {'.exe', '.pdf', '.zip', '.rar', '.jpg', '.png', '.doc', '.docx', '.xlsx', '.txt'}

seen = set()

for csv_path in df_csvs['chats'].dropna().unique():
    if isinstance(csv_path, str) and csv_path.endswith('.csv') and os.path.exists(csv_path):
        try:
            df = pd.read_csv(csv_path)
        except Exception as e:
            print(f"Error reading {csv_path}: {e}")
            continue

        df.columns = [col.lower() for col in df.columns]
        text_cols = [col for col in df.columns if 'message' in col]

        for idx, row in df.iterrows():
            for col in text_cols:
                if pd.notna(row.get(col)):
                    text = str(row[col])

                    for email in email_pattern.findall(text):
                        key = ("email", email, csv_path, idx)
                        if key not in seen:
                            seen.add(key)
                            print(f"[EMAIL] {email} | File: {os.path.basename(csv_path)} | Line: {idx}")

                    for ip in ip_pattern.findall(text):
                        key = ("ip", ip, csv_path, idx)
                        if key not in seen:
                            seen.add(key)
                            print(f"[IP] {ip} | File: {os.path.basename(csv_path)} | Line: {idx}")

                    for domain in domain_pattern.findall(text):
                        ext = '.' + domain.split('.')[-1].lower()
                        if '@' not in domain and ext not in bad_extensions:
                            key = ("domain", domain, csv_path, idx)
                            if key not in seen:
                                seen.add(key)
                                print(f"[DOMAIN] {domain} | File: {os.path.basename(csv_path)} | Line: {idx}")

                    if isoon_pattern.search(text):
                        key = ("isoon", text, csv_path, idx)
                        if key not in seen:
                            seen.add(key)
                            print(f"[I-SOON] {text} | File: {os.path.basename(csv_path)} | Line: {idx}")

[DOMAIN] 7e198208-6603-438d-836d-d24a47670b8d.md | File: 18.csv | Line: 31
[DOMAIN] 5c84f2e2-414e-4716-bc96-0218b1bc964f.md | File: 1.csv | Line: 10
[DOMAIN] 4b32e3c0-0887-49b2-a164-aac04b1d3da4.md | File: 1.csv | Line: 415
[DOMAIN] meeting.tencent.com | File: 1.csv | Line: 546
[DOMAIN] 351180cd-d982-40a2-8881-8f954cae8fd4.md | File: 1.csv | Line: 821
[DOMAIN] 28d70cf7-5244-4834-8169-d491781f260c.md | File: 1.csv | Line: 1334
[DOMAIN] 1188a898-edb0-40f9-a05c-d59664095e3b.md | File: 1.csv | Line: 1335
[DOMAIN] 671ed94d-c7a1-4063-923d-9765780b8f78.md | File: 1.csv | Line: 1336
[DOMAIN] 8a0eecb1-519e-4e8c-bfba-47ceb043943f.md | File: 1.csv | Line: 1384
[DOMAIN] www.yutainews.com | File: 1.csv | Line: 1671
[DOMAIN] 16079.html | File: 1.csv | Line: 1671
[DOMAIN] finance.sina.com.cn | File: 1.csv | Line: 1681
[DOMAIN] doc-iivhvpwx8123040.shtml | File: 1.csv | Line: 1681
[DOMAIN] Sina.com | File: 1.csv | Line: 1897
[DOMAIN] 86ec836a-b3a4-4c4b-9906-29c99481ea75.md | File: 1.csv | Line: 2121
[D

In [24]:
def extract_unique_users_from_csv(csv_path):
    try:
        df = pd.read_csv(csv_path)
    except Exception as e:
        print(f"Error reading {csv_path}: {e}")
        return set()

    # Try to find user-related columns
    user_columns = [col for col in df.columns if col.lower() in ['from', 'to', 'sender', 'receiver', 'user']]
    
    if not user_columns:
        user_columns = df.select_dtypes(include=['object']).columns.tolist()

    # Flatten and clean user list
    users = pd.unique(df[user_columns].values.ravel())
    return {str(u).strip() for u in users if pd.notna(u) and str(u).strip() != ''}

In [25]:
all_users = set()

# Loop through each file path in 'chats' column
for csv_path in df_csvs_llm['chats'].dropna().unique():
    if isinstance(csv_path, str) and csv_path.endswith('.csv') and os.path.exists(csv_path):
        users = extract_unique_users_from_csv(csv_path)
        all_users.update(users)

unique_users_from_chats = sorted(all_users)
print("Unique users found in chat logs:")
for user in unique_users_from_chats:
    print("-", user)

Unique users found in chat logs:
- 25713010771@chatroom
- SWEET5683yao
- Shutd0wn
- adpw90
- dujijiyiqxx
- gzp1991101
- hack05112
- ibabaimama
- just910420
- ken73224
- lengmo
- nullroot
- qq78263462
- snipersk
- tianyi-0608
- wangchao953541
- wei592628
- wxid_12n748um1thl21
- wxid_5390224027312
- wxid_70w3p1jin84k22
- wxid_7p054rmzkhqf21
- wxid_blw54o1q0q5w22
- wxid_c9yv0nsla3yn22
- wxid_hlmnhsq64tt722
- wxid_icges6alg8cl21
- wxid_jcnxegjccqi441
- wxid_kbys0kvzj4ta12
- wxid_mgh25nentc4u22
- wxid_nv9bv435fz3722
- wxid_q7vkst94g5u011
- wxid_soekgggwnfgm21
- wxid_wh6x59w70y3r22
- wxid_xusilpfkh31g21
- wxid_zb45i0rc71yk21
- wxid_zbytkn4qjl3r22
- yanzi542766277
- zhangxiaoyan0422


# **LLM extraction of user relevant data for identification**

In [14]:
import pandas as pd
import re

# Load the chat log
df = pd.read_csv("csvs/1.csv")
df = df.dropna(subset=["message_translation"])

# Keywords related to meeting invitations 
keywords = [
    "meeting", "conference", "webinar", "call", "join", "schedule", "host", "invites you",
    "meeting id", "meeting link", "meeting password", "join the meeting",
    "click the link", "click to join", "meeting url", "one-click dial-in", "dial-in number",
    "passcode", "topic", "subject", "time", "hosted by", "organizer",
    "会议", "会议主题", "会议链接", "会议时间", "会议密码", "会议ID",
    "邀请", "主持人", "加入会议", "点击链接", "参会", "入会", "参会链接", "密码", "时间",
    "tencent meeting", "meeting.tencent.com", "腾讯会议",
    "dingtalk", "钉钉",
    "feishu", "飞书", "lark",
    "zoom", "wecom", "企业微信",
    "voov", "voov meeting", "meeting.voov.com"
]

# Create a pattern to search for those keywords
pattern = re.compile("|".join(re.escape(word) for word in keywords), re.IGNORECASE)

# Check if a message is likely a meeting invite
def is_meeting_message(text):
    return len(re.findall(pattern, text)) >= 2

# Look through each message
for index, row in df.iterrows():
    msg = row["message_translation"]
    if is_meeting_message(msg):
        print(f"Row: {index}")
        print(f"From: {row['From']} → To: {row['To']}")
        print(f"Message: {msg[:1000]}")
        print("-" * 60)

Row: 546
From: Shutd0wn → To: lengmo
Message: Wu Haibo invites you to a Tencent Meeting
Meeting Topic: Meeting scheduled by Wu Haibo
Meeting Time: 2020/8/29 14:00-15:30

Click the link to join, or add to meeting list:
https://meeting.tencent.com/s/4zEAlh2Uoc6x

Meeting ID: 612 649 561
Meeting Password: 111222

One-click dial-in from your phone
+8675536550000,,612649561# (Mainland China)
+85230018898,,,2,612649561# (Hong Kong)

Dial according to your location
+8675536550000 (Mainland China)
+85230018898 (Hong Kong)
------------------------------------------------------------
Row: 1564
From: lengmo → To: Shutd0wn
Message: However, from a financial perspective, we really can't allow this to be completely open-ended. I remember there being a taxi approval process in DingTalk, which also prevents people from taking taxis unnecessarily. From now on, all taxi requests must be approved through DingTalk.
------------------------------------------------------------
Row: 3357
From: lengmo → To: S

In [36]:
import pandas as pd
import re

# Load and clean data
df = pd.read_csv("csvs/1.csv")
df = df.dropna(subset=["message_translation"])

# Account-related keywords
account_keywords = [
    "username", "login", "user", "account", "access", "credentials",
    "password", "passcode", "auth", "key", "token", "session",
    "bank", "iban", "swift", "wallet", "crypto", "btc", "wallet address",
    "支付宝", "微信支付", "账号", "账户", "密码", "卡号", "登录", "银行"
]

# Add Chinese bank names
chinese_banks = [
    "中国银行", "建设银行", "工商银行", "农业银行", "交通银行", "招商银行", 
    "邮储银行", "民生银行", "中信银行", "浦发银行", "兴业银行", "华夏银行",
    "广发银行", "平安银行", "北京银行", "上海银行", "光大银行", "渤海银行"
]

# Combine all terms
all_keywords = account_keywords + chinese_banks

# Compile regex pattern
pattern = re.compile("|".join(re.escape(word) for word in all_keywords), re.IGNORECASE)

# Define match condition
def is_account_message(text):
    return len(re.findall(pattern, text)) >= 2

# Scan and print results
for index, row in df.iterrows():
    msg = row["message_translation"]
    if is_account_message(msg):
        print(f"Row: {index}")
        print(f"From: {row['From']} → To: {row['To']}")
        print(f"Message: {msg[:2300]}")
        print("-" * 60)

Row: 1669
From: lengmo → To: Shutd0wn
Message: However, according to a local court judgment, Beijing Ruibang is suspected of bribery. According to a judgment from the Intermediate People's Court of Sanming City, Fujian Province, Beijing Ruibang once had procurement dealings worth tens of millions of yuan with a local network security brigade, and Beijing Ruibang once provided up to 1.188 million yuan in “sponsorship fees” to relevant officials of the local public security organs.
------------------------------------------------------------
Row: 3209
From: lengmo → To: Shutd0wn
Message: For 2021, will key accounts also participate in assessments and commissions like the sales department?
------------------------------------------------------------
Row: 3211
From: lengmo → To: Shutd0wn
Message: From now on, any clients I sign will be under the Key Account Department, and the General Office won't have any performance requirements.
----------------------------------------------------------

In [27]:
import pandas as pd
import re

# Load and clean data
df = pd.read_csv("csvs/1.csv")
df = df.dropna(subset=["message_translation"])

# List of hierarchical or role-related terms
role_keywords = [
    # Leadership
    "leader", "admin", "supervisor", "manager", "director", "chief", "commander", "boss",

    # Technical
    "developer", "coder", "engineer", "technician", "analyst", "pentester", "red team", "blue team", "researcher",

    # Operational
    "operator", "handler", "organizer", "facilitator", "coordinator", "liaison", "logistics",

    # Recruitment & support
    "recruiter", "consultant", "advisor", "insider",

    # Financial/Criminal
    "money mule", "courier", "broker", "affiliate", "partner", "accomplice", "funder", "supplier", "investor", "fence", "associate"
]

# Compile a case-insensitive regex
role_pattern = re.compile(r"\b(" + "|".join(re.escape(term) for term in role_keywords) + r")\b", re.IGNORECASE)

# Function to match role terms
def is_role_message(text):
    return bool(role_pattern.search(text))

# Scan and print matches
for index, row in df.iterrows():
    msg = row["message_translation"]
    if is_role_message(msg):
        print(f"Row: {index} | From: {row['From']} → To: {row['To']}")
        print(f"Message: {msg[:1300]}")
        print("-" * 60)

[ROLE] Row: 158 | From: lengmo → To: Shutd0wn
Message: Previously, a department director from the Political and Legal Committee, at the end of August and beginning of September, is going to a county to serve as the county head... I'm planning to collaborate with that county.
------------------------------------------------------------
[ROLE] Row: 180 | From: lengmo → To: Shutd0wn
Message: Now the director of the Provincial Political and Legal Committee is said to be going down to gain experience.
------------------------------------------------------------
[ROLE] Row: 232 | From: lengmo → To: Shutd0wn
Message: Chief Ecological Officer
------------------------------------------------------------
[ROLE] Row: 624 | From: Shutd0wn → To: lengmo
Message: Regarding this matter, please arrange a time to come to Shanghai, bring along 村长 (Village Chief) to discuss the details of 东风 (Dongfeng), and also bring 小潘 (Xiao Pan). Then this matter will be settled.
---------------------------------------

In [28]:
import pandas as pd
import re
import json

# Load chat log
df = pd.read_csv("csvs/1.csv")
df = df.dropna(subset=["message_translation"]).reset_index(drop=True)

# Function to enrich messages with ±3 context
def enrich_messages(df, indices, window=3):
    enriched = []
    for idx in sorted(set(indices)):
        start = max(0, idx - window)
        end = min(len(df), idx + window + 1)
        context_rows = df.iloc[start:end]

        enriched.append({
            "focus_index": idx,
            "context": [
                {
                    "timestamp": row["Time"],
                    "from": row["From"],
                    "to": row["To"],
                    "message": row["message_translation"]
                }
                for _, row in context_rows.iterrows()
            ]
        })
    return enriched

In [29]:
def detect_meetings(df):
    keywords = [
        "meeting", "conference", "webinar", "call", "join", "schedule", "host", "invites you",
        "meeting id", "meeting link", "meeting password", "join the meeting",
        "click the link", "click to join", "meeting url", "one-click dial-in", "dial-in number",
        "passcode", "topic", "subject", "time", "hosted by", "organizer",
        "会议", "会议主题", "会议链接", "会议时间", "会议密码", "会议ID",
        "邀请", "主持人", "加入会议", "点击链接", "参会", "入会", "参会链接", "密码", "时间",
        "tencent meeting", "meeting.tencent.com", "腾讯会议",
        "dingtalk", "钉钉", "feishu", "飞书", "lark", "zoom", "wecom", "企业微信",
        "voov", "voov meeting", "meeting.voov.com"
    ]
    pattern = re.compile("|".join(re.escape(k) for k in keywords), re.IGNORECASE)
    return [i for i, m in enumerate(df["message_translation"]) if len(re.findall(pattern, m)) >= 2]

def detect_accounts(df):
    keywords = [
        "username", "login", "user", "account", "access", "credentials",
        "password", "passcode", "auth", "key", "token", "session",
        "bank", "iban", "swift", "wallet", "crypto", "btc", "wallet address",
        "支付宝", "微信支付", "账号", "账户", "密码", "卡号", "登录", "银行",
        "中国银行", "建设银行", "工商银行", "农业银行", "交通银行", "招商银行", 
        "邮储银行", "民生银行", "中信银行", "浦发银行", "兴业银行", "华夏银行",
        "广发银行", "平安银行", "北京银行", "上海银行", "光大银行", "渤海银行"
    ]
    pattern = re.compile("|".join(re.escape(k) for k in keywords), re.IGNORECASE)
    return [i for i, m in enumerate(df["message_translation"]) if len(re.findall(pattern, m)) >= 2]

def detect_roles(df):
    roles = [
        "leader", "admin", "supervisor", "manager", "director", "chief", "commander", "boss",
        "developer", "coder", "engineer", "technician", "analyst", "pentester", "red team", "blue team", "researcher",
        "operator", "handler", "organizer", "facilitator", "coordinator", "liaison", "logistics",
        "recruiter", "consultant", "advisor", "insider",
        "money mule", "courier", "broker", "affiliate", "partner", "accomplice", "funder", "supplier", "investor", "fence", "associate"
    ]
    pattern = re.compile(r"\b(" + "|".join(re.escape(r) for r in roles) + r")\b", re.IGNORECASE)
    return [i for i, m in enumerate(df["message_translation"]) if pattern.search(m)]

In [30]:
# Get all matching indices
indices = (
    detect_meetings(df) +
    detect_accounts(df) +
    detect_roles(df)
)

# Enrich context
enriched = enrich_messages(df, indices)

# Save to file
with open("enriched_output_1.csv.json", "w", encoding="utf-8") as f:
    json.dump(enriched, f, ensure_ascii=False, indent=2)

print(f"Saved {len(enriched)} enriched entries to enriched_output.json")

Saved 243 enriched entries to enriched_output.json


In [46]:
import json
import requests
from collections import defaultdict

# === Config ===
json_path = "enriched_output_1.csv.json"
model = "qwen3:32b"  # Adjust to your local model

# === Load full enriched data ===
with open(json_path, "r", encoding="utf-8") as f:
    enriched_data = json.load(f)

# === Group messages by sender across all blocks ===
user_messages = defaultdict(list)

for block in enriched_data:
    for msg in block["context"]:
        sender = msg["from"]
        user_messages[sender].append(
            f"[{msg['timestamp']}] {msg['from']} → {msg['to']}:\n{msg['message']}"
        )

# === Create prompt for one user based on all their messages ===
def create_full_prompt(user, messages):
    messages_text = "\n".join(messages)
    return f"""
You are a law enforcement officer analysing some chat logs

Below is a collection of internal messages written by a user named "{user}". Your task is to create a structured profile of this person.

ONLY ANSWER THE FOLLOWING in this format and include only 1 answer + give confidence:
Username: <username>
Real name: <real name>
Working for: <company name>
Position in the Company: <company position>

Messages:
{messages_text}

End of messages.
Generate the profile for: {user}
""".strip()

# === Loop through each sender, send prompt to Ollama ===
for user, messages in user_messages.items():
    print(f"\n\n==== PROFILE FOR: {user} ====\n")

    prompt = create_full_prompt(user, messages)

    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": model, "prompt": prompt, "stream": False}
    )

    print(response.json()["response"])
    print("=" * 80)



==== PROFILE FOR: lengmo ====

<think>
Okay, let's start by looking at the user's query. They want me to generate a profile for "lengmo" based on the provided message history. First, I need to parse through all the messages to identify key information about Lengmo's role, responsibilities, interactions, and the context in which they operate.

Looking at the messages, Lengmo frequently communicates with colleagues about various projects. The projects mentioned include IT projects like integrated security, telecom fraud cases, and projects in different cities like Yangzhou, Jincheng, Jingdezhen, etc. This suggests that Lengmo is likely involved in project management or a role that oversees multiple projects, possibly in a technical or IT company.

Lengmo mentions terms like "preliminary design document," "acceptance application," "outsourced components," and "case clues." These terms are common in project management or technical implementation. The mention of financial aspects like "in

KeyboardInterrupt: 

# **Financial Infrastructures Analysis**

In [2]:
from langchain_ollama.llms import OllamaLLM
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
import pandas as pd
import os

In [3]:
import os
import pandas as pd
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.schema import Document

# Load a single CSV file
csv_file = "csvs/10.csv"
df = pd.read_csv(csv_file)

# Initialize the embedding model
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

# Setup vector DB path
db_location = "./chroma_financial_chat_db"
add_documents = not os.path.exists(db_location)

# Only add documents if the database doesn't already exist
if add_documents:
    documents = []
    ids = []
    for i, row in df.iterrows():
        document = Document(
            page_content=row["message_translation"],
            metadata={
                "sender": row["From"],
                "receiver": row["To"],
                "timestamp": row["Time"],
                "source_file": os.path.basename(csv_file),   # Track source file
                "line_number": i + 1                         # Track line number
            },
            id=f"{os.path.basename(csv_file)}_{i}"            # Ensure unique ID
        )
        documents.append(document)
        ids.append(f"{os.path.basename(csv_file)}_{i}")

# Create or load the Chroma vector store
vector_store = Chroma(
    collection_name="financial_chats",
    persist_directory=db_location,
    embedding_function=embeddings
)

if add_documents:
    vector_store.add_documents(documents=documents, ids=ids)

# Setup retriever for use in RAG
retriever = vector_store.as_retriever(search_kwargs={"k": 15})

/tmp/ipykernel_20294/3807150908.py:38: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vector_store = Chroma(


In [4]:
model = OllamaLLM(model="gemma3:27b") 

template = """
You are a cybersecurity analyst reviewing translated chat messages for financial intelligence.

Your task is to extract ONLY the messages that contain **financially relevant information**, such as:

- Payments or money transfers
- Amounts, currencies, or sums of money
- Payment methods (cash, crypto, bank, wallets)
- Account numbers, codes, or financial identifiers
- Mentions of financial roles (payer, payee, client, broker)
- References to suspicious or recurring financial activity

From the messages below:
{chats}

Return ONLY the messages that meet any of the criteria above.

For each relevant message, output:
- The **timestamp** (if provided in the metadata or inferred)
- The **translated content** of the message

Do not summarize. Do not explain. Do not include non-financial content.  
**DO NOT create or infer messages. Use only the exact text from the messages provided. If no messages match, return nothing.**

Output format:
[Time] Message
"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model

In [5]:
question = "Extract only the chat messages that refer to money, payments, currencies, accounts, financial identifiers, or financial discussions"

# Retrieve relevant chats
retrieved_docs = retriever.invoke(question)

# Combine messages with metadata for context
chat_texts = "\n\n".join([
    f"[{doc.metadata['source_file']} - line {doc.metadata['line_number']}] {doc.page_content}"
    for doc in retrieved_docs
])

# Run through the LLM
response = chain.invoke({"chats": chat_texts})

# Output the result
print(response)

[10.csv - line 21] Account TGtadie, phone: 18510867099, name: Wang Ning.




